In [1]:
import export_objects as eo
import analyze_subroutines as alsub
import fortran_modules as fm
import utilityFunctions as uf
import importlib
import os
import csv

import database.connect as cnt
import database.utilities as util

importlib.reload(eo)
importlib.reload(alsub)
importlib.reload(fm)
importlib.reload(uf)
importlib.reload(cnt)
importlib.reload(util)

mod_dict, sub_dict, type_dict = {}, {}, {}
mod_dict, sub_dict, type_dict = eo.unpickle_unit_test(mod_dict, sub_dict, type_dict)


SELECT subroutines.subroutine_name,user_type_instances.instance_name,subroutine_active_global_vars.status,type_definitions.member_name FROM subroutines JOIN user_type_instances JOIN subroutine_active_global_vars JOIN type_definitions WHERE subroutine_active_global_vars.subroutine_id = subroutines.subroutine_id AND user_type_instances.instance_id = subroutine_active_global_vars.instance_id AND subroutine_active_global_vars.member_id = type_definitions.define_id


In [3]:
sub_dict['laketemperature'].analyze_calltree(sub_dict['laketemperature'].calltree[2:], 'info_files')

laketemperature
|---->soilthermprop_lake
|---->tridiagonal_sr
|---->phasechange_lake


In [28]:
subroutine_list = {'soiltemperature': sub_dict['soiltemperature']}
modtree = fm.print_spel_module_dependencies(mod_dict=mod_dict,subs=subroutine_list)

# Print the module tree
arrow = "-->"
cutoff_depth = 10 # Only print modules up to this depth
suppress_mod_list = ['shr_kind_mod']
for m in modtree:
    depth = m['depth']
    modname = m['module']
    if(modname in suppress_mod_list):
        continue
    if(depth == 1):
        print(arrow*depth + modname )
    elif(depth <= cutoff_depth):
        print( arrow*depth + modname)

/mnt/c/Users/mungs/Desktop/SPEL_OpenACC/scripts/../../repo/E3SM/components/elm/src/biogeophys/SoilTemperatureMod.F90
1:module SoilTemperatureMod
soiltemperaturemod
-->decompmod
-->-->elm_varctl
-->-->elm_varcon
-->-->-->shr_const_mod
-->-->-->elm_varpar
-->-->-->-->elm_varctl
-->-->-->elm_varctl
-->-->domainmod
-->-->-->elm_varctl
-->elm_varctl
-->elm_varcon
-->-->shr_const_mod
-->-->elm_varpar
-->-->-->elm_varctl
-->-->elm_varctl
-->urbanparamstype
-->-->decompmod
-->-->-->elm_varctl
-->-->-->elm_varcon
-->-->-->-->shr_const_mod
-->-->-->-->elm_varpar
-->-->-->-->-->elm_varctl
-->-->-->-->elm_varctl
-->-->-->domainmod
-->-->-->-->elm_varctl
-->-->elm_varctl
-->-->elm_varcon
-->-->-->shr_const_mod
-->-->-->elm_varpar
-->-->-->-->elm_varctl
-->-->-->elm_varctl
-->-->landunittype
-->-->-->elm_varcon
-->-->-->-->shr_const_mod
-->-->-->-->elm_varpar
-->-->-->-->-->elm_varctl
-->-->-->-->elm_varctl
-->-->gridcelltype
-->-->-->landunit_varcon
-->-->-->elm_varcon
-->-->-->-->shr_const_mod
-->

In [16]:
sub_dict['banddiagonal'].child_Subroutine['dgbsv'].print_subroutine_info()



Subroutine dgbsv in lib.F90 L-999--999
Has Arguments:
Local Arrays:


True

In [33]:
# connect to database
cur, conn = cnt.connect()

# use file to create csv for module table
csv_folder = "CSV"
info_files = "info_files"
os.makedirs(csv_folder, exist_ok=True)
os.makedirs(info_files, exist_ok=True)

In [100]:
# write modules into to file
def write_modules_info_to_text(filename):
    with open(filename, "w") as file:
        for keys in mod_dict.keys():
            mod_dict[keys].export_module_info(ofile=file)
            
# write subroutines into to file
def write_subroutines_info_to_text(filename):
    with open(filename, "w") as file:
        for i in sub_dict.keys():
            sub_dict[i].export_subroutine_info(ofile=file)
            
# write types into to file
def write_types_info_to_text(filename):
    with open(filename, "w") as file:
        for i in type_dict.keys():
            type_dict[i].export_derived_type(ofile=file)

In [ ]:
write_modules_info_to_text(f"{info_files}/modules.txt")
write_subroutines_info_to_text(f"{info_files}/subroutine.txt")
write_types_info_to_text(f"{info_files}/types_info.txt")

In [27]:


def TABLE_modules(filename, csv_file="TABLE_module"):
    
    """
    TABLE SETUP:
    +-------------+--------------+------+-----+---------+----------------+
    | Field       | Type         | Null | Key | Default | Extra          |
    +-------------+--------------+------+-----+---------+----------------+
    | module_id   | int          | NO   | PRI | NULL    | auto_increment |
    | module_name | varchar(100) | NO   | UNI | 0       |                |
    +-------------+--------------+------+-----+---------+----------------+
    
    INITIALIZE NULL WITH COMMAND
    
    INSERT INTO modules (module_name) VALUES ('NULL');
    UPDATE modules SET module_id=0 where module_id=1;
    
    """

    cols = ["module_name"]
    
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        
        with open(filename, "r") as reader:
            for line in reader:
                if "Module:" in line:
                    # Module: <module>
                    module = line.split(" ")[-1].strip()
                    
                    writer.writerow([module])

def TABLE_module_dependency(filename, csv_file="TABLE_module_dependency"):
    
    """
    TABLE SETUP
    +---------------+--------------+------+-----+---------+----------------+
    | Field         | Type         | Null | Key | Default | Extra          |
    +---------------+--------------+------+-----+---------+----------------+
    | dependency_id | int          | NO   | PRI | NULL    | auto_increment |
    | module_id     | int          | YES  | MUL | NULL    |                |
    | dep_module_id | int          | YES  | MUL | NULL    |                |
    | object_used   | varchar(100) | YES  |     | NULL    |                |
    +---------------+--------------+------+-----+---------+----------------+
    """
    
    cols = ["module_id", "dep_module_id","object_used"]

    with open(f"{csv_folder}/{csv_file}.csv","w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, "r") as reader:
            cur_module = ""
            cur_dep = ""
            for line in reader:
                if "Module:" in line:
                    # Module: <module>
                    cur_module = line.split(" ")[-1].strip()
                elif "->" in line:
                    if "None" in line:
                        writer.writerow([cur_module, "NULL", "NULL"])
                        continue
                    cur_dep = line.split(" ")[-1].strip()
                elif line[:4] == "   -":
                    line = line.strip().split()[1:]
                    if "=>" in line:
                        line = line[2:]
                    if len(line) == 1:
                        writer.writerow([cur_module, cur_dep, line[0].strip()])

                    if len(line) == 3:
                        writer.writerow([cur_module, cur_dep, line[1].strip()])
                    

In [30]:
filename = f"{info_files}/modules.txt"

TABLE_modules(filename)
TABLE_module_dependency(filename)

util.insert(cur,
            conn,
            "modules",
            "CSV/TABLE_module.csv")

util.insert(cur, 
            conn, 
            "module_dependency", 
            "CSV/TABLE_module_dependency.csv", 
            join=True, 
            ref_tables=["modules", "modules", 2],
            selections=["module_id","module_id",""],
            conditions=["module_name","module_name"],
            dependency=[0,1,None])

In [ ]:
def TABLE_user_types(filename, csv_file='TABLE_user_types'):
    """
    TABLE SETUP
    +----------------+--------------+------+-----+---------+----------------+
    | Field          | Type         | Null | Key | Default | Extra          |
    +----------------+--------------+------+-----+---------+----------------+
    | user_type_id   | int          | NO   | PRI | NULL    | auto_increment |
    | module_id      | int          | NO   | MUL | 0       |                |
    | user_type_name | varchar(100) | NO   | UNI | 0       |                |
    +----------------+--------------+------+-----+---------+----------------+
    """
    cols = ["module_id", "user_type_name"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, 'r') as f:
            type = ""
            mod = ""
            for line in f:
                if "Type:" in line:
                    type = line.split()[-1].strip()
                elif "Mod:" in line:
                    mod = line.split()[-1].strip()
                    writer.writerow([mod, type])

def TABLE_user_type_instances(filename, csv_file='TABLE_user_type_instances'):
    
    """
    TABLE SETUP
    +------------------+--------------+------+-----+---------+----------------+
    | Field            | Type         | Null | Key | Default | Extra          |
    +------------------+--------------+------+-----+---------+----------------+
    | instance_id      | int          | NO   | PRI | NULL    | auto_increment |
    | instance_type_id | int          | NO   | MUL | 0       |                |
    | instance_name    | varchar(100) | NO   |     |         |                |
    +------------------+--------------+------+-----+---------+----------------+
    """
    
    cols = ["instance_type_id", "instance_name"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, 'r') as f:
            type = ""
            
            for line in f:
                if "Type:" in line:
                    type = line.split()[-1].strip()
                elif "i:" in line:
                    instance = line.split()[2].strip()
                    writer.writerow([type, instance])

In [162]:
filename = f"{info_files}/types_info.txt"

TABLE_user_types(filename)
TABLE_user_type_instances(filename)

util.insert(cur, 
            conn, 
            "user_types", 
            "CSV/TABLE_user_types.csv", 
            join=True, 
            ref_tables=["modules", 1],
            selections=["module_id", ""],
            conditions=["module_name", ""],
            dependency=[0,None])

util.insert(cur,
            conn,
            "user_type_instances",
            "CSV/TABLE_user_type_instances.csv",
            join=True,
            ref_tables=["user_types", 1],
            selections=["user_type_id", ""],
            conditions=['user_type_name', ""],
            dependency=[0,None])

In [199]:
def TABLE_type_defines(filename, csv_file="TABLE_type_defines"):
    with open(csv_file, 'w') as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(['subroutine_parent', "module_parent", "subroutine_child", "module_child", "arg"])

In [36]:
def TABLE_subroutines(filename, csv_file="TABLE_subroutines"):
    """
    TABLE SETUP
    +-----------------+--------------+------+-----+---------+----------------+
    | Field           | Type         | Null | Key | Default | Extra          |
    +-----------------+--------------+------+-----+---------+----------------+
    | subroutine_id   | int          | NO   | PRI | NULL    | auto_increment |
    | subroutine_name | varchar(100) | NO   | UNI | 0       |                |
    | module_id       | int          | NO   | MUL | 0       |                |
    +-----------------+--------------+------+-----+---------+----------------+
    """
    cols = ["subroutine_name", "module_id"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        
        with open(filename, "r") as r:
            sub = ""
            mod = ""
            for line in r:
                if "Sub:" in line:
                   sub = line.split()[-1].strip() 
                elif "Mod:" in line:
                    mod = line.split()[-1].strip()
                    writer.writerow([sub, mod])

def TABLE_subroutines_args(filename, csv_file="TABLE_subroutine_args"):
    """
    TABLE SETUP
    +---------------+--------------+------+-----+---------+----------------+
    | Field         | Type         | Null | Key | Default | Extra          |
    +---------------+--------------+------+-----+---------+----------------+
    | arg_id        | int          | NO   | PRI | NULL    | auto_increment |
    | subroutine_id | int          | NO   | MUL | 0       |                |
    | arg_type      | varchar(100) | NO   |     | 0       |                |
    | arg_name      | varchar(100) | NO   |     | 0       |                |
    | dim           | int          | NO   |     | 0       |                |
    +---------------+--------------+------+-----+---------+----------------+
    """
    cols = ["subroutine_id", "arg_type", "arg_name", "dim"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        
        with open(filename, "r") as r:
            sub = ""
            for line in r:
                if "Sub:" in line:
                   sub = line.split()[-1].strip() 
                elif "-" in line:
                    if "None" in line:
                        writer.writerow([sub, "Null", "Null", "-1"])
                        continue
                    line = line.split()
                    line = [l.strip() for l in line]
                 
                    writer.writerow([sub, line[1], line[-2].strip("'"), line[-1]])

def TABLE_subroutines_calltree(filename, csv_file="TABLE_subroutine_calltree"):
    """
    TABLE SETUP
    +-----------------------+------+------+-----+---------+----------------+
    | Field                 | Type | Null | Key | Default | Extra          |
    +-----------------------+------+------+-----+---------+----------------+
    | parent_id             | int  | NO   | PRI | NULL    | auto_increment |
    | parent_subroutine_id  | int  | NO   | UNI | 0       |                |
    | child_subroutine_id   | int  | NO   | MUL | 0       |                |
    +-----------------------+------+------+-----+---------+----------------+
    """
    cols = ["parent_subroutine_id", "child_subroutine_id"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        
        with open(filename, "r") as r:
            sub = ""
            for line in r:
                if "ParentSub:" in line:
                   sub = line.split()[-1].strip() 
                elif "c:" in line:
                    if "None" in line:
                        writer.writerow([sub, "Null"])
                        continue
                    else:
                        line = line.split()[-1].strip()
                    
                    
                        writer.writerow([sub, line])



def TABLE_subroutine_active_global_vars(filename= "CSV/_TABLE_subroutine_active_global_vars.csv",
                                        csv_file="TABLE_subroutine_active_global_vars"):
    """
    TABLE SETUP
    +---------------+---------+------+-----+---------+----------------+
    | Field         | Type    | Null | Key | Default | Extra          |
    +---------------+---------+------+-----+---------+----------------+
    | variable_id   | int     | NO   | PRI | NULL    | auto_increment |
    | subroutine_id | int     | NO   | MUL | 0       |                |
    | instance_id   | int     | YES  | MUL | NULL    |                |
    | member_id     | int     | NO   | MUL | 0       |                |
    | status        | char(2) | NO   |     | 0       |                |
    +---------------+---------+------+-----+---------+----------------+
    """
    eo.create_dataframe(sub_dict, "CSV/_TABLE_subroutine_active_global_vars")
    
    cols = ["subroutine_id", "instance_id", "member_id", "status"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, "r") as r:
            header = True
            reader = csv.reader(r)
            for line in reader:
                if header:
                    header = False
                    continue
                # print(line)
                line = [l.strip() for l in line]
                var = line[2].split('%')
                instance = var[0]
                member = var[1]
                writer.writerow([line[1], instance, member, line[3]])

def TABLE_subroutines_local_array(filename, csv_file="TABLES_subroutine_local_array"):
    """
    TABLE SETUP
    +---------------+--------------+------+-----+---------+----------------+
    | Field         | Type         | Null | Key | Default | Extra          |
    +---------------+--------------+------+-----+---------+----------------+
    | local_arry_id | int          | NO   | PRI | NULL    | auto_increment |
    | subroutine_id | int          | NO   | MUL | 0       |                |
    | array_name    | varchar(100) | NO   |     | 0       |                |
    | dim           | int          | NO   |     | 0       |                |
    +---------------+--------------+------+-----+---------+----------------+
    """
    cols = ["subroutine_id", "array_name", "dim"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        
        with open(filename, "r") as r:
            sub = ""
            for line in r:
                if "ParentSub:" in line:
                   sub = line.split()[-1].strip() 
                elif "l:" in line:
                    if "None" in line:
                        writer.writerow([sub, "Null", "-1"])
                        continue
                    else:
                        line = line.split()
                        line = [l.strip() for l in line]
                    
                    
                        writer.writerow([sub, line[2], line[-1]])

In [9]:
TABLE_subroutines_calltree()
util.insert(cur,
            conn,
            "subroutine_calltree",
            "CSV/TABLE_subroutine_calltree.csv",
            join=True,
            ref_tables=['subroutines', 'subroutines'],
            selections=['subroutine_id', 'subroutine_id'],
            conditions=['subroutine_name', 'subroutine_name'],
            dependency=[0, 1])


['banddiagonal', 'dgbsv']


In [73]:
import export_objects as eo
import analyze_subroutines as alsub
import fortran_modules as fm
import utilityFunctions as uf
import importlib
import os
import csv

import database.connect as cnt
import database.utilities as util
# 
importlib.reload(eo)
importlib.reload(alsub)
importlib.reload(fm)
importlib.reload(uf)
importlib.reload(cnt)
importlib.reload(util)
#
mod_dict, sub_dict, type_dict = {}, {}, {}
mod_dict, sub_dict, type_dict = eo.unpickle_unit_test(mod_dict, sub_dict, type_dict)
cur, conn = cnt.connect()


In [74]:


# TABLE_subroutine_active_global_vars()
util.insert(cur,
            conn,
            "subroutine_active_global_vars",
            "CSV/TABLE_subroutine_active_global_vars.csv",
            join=True,
                # cols = ["subroutine_id", "instance_id", "member_id", "status"]
            ref_tables=['subroutines', "user_type_instances", 'type_definitions', 3],
            selections=['subroutine_id', 'instance_id', "define_id", ""],
            conditions=['subroutine_name', 'instance_name', 'member_name'],
            dependency=[0, 1, 2, None],
            extra=["m1.instance_type_id=m2.user_type_id"])

['soiltemperature', 'veg_ef', 'dgnetdt', 'rw']
['computegroundheatfluxandderiv', 'veg_ef', 'dgnetdt', 'rw']
['surfacealbedo', 'ldomain', 'pftm', 'r']
['soilalbedo', 'ldomain', 'pftm', 'r']


In [42]:
filename = f"{info_files}/subroutine.txt"

TABLE_subroutines(filename)
TABLE_subroutines_args(filename)
TABLE_subroutines_calltree(filename)
TABLE_subroutine_active_global_vars()
TABLE_subroutines_local_array(filename)

util.insert(cur,
            conn,
            "subroutines",
            "CSV/TABLE_subroutines.csv",
            join=True,
            ref_tables=[0, "modules"],
            selections=["","module_id"],
            conditions=["","module_name"],
            dep_cols=[None,1])

util.insert(cur,
            conn,
            "subroutine_args",
            "CSV/TABLE_subroutine_args.csv",
            join=True,
            ref_table=["subroutines", 1, 2, 3],
            selections=["subroutine_id", "", "", ""],
            conditions=["subroutine_name", "", "", ""],
            dependency=[0, None, None, None])

util.insert(cur,
            conn,
            "subroutine_calltree",
            "CSV/TABLE_subroutine_calltree.csv",
            join=True,
            ref_tables=['subroutines', 'subroutines'],
            selections=['subroutine_id', 'subroutine_id'],
            conditions=['subroutine_name', 'subroutine_name'],
            dependency=[0, 1])

util.insert(cur,
            conn,
            "subroutine_active_global_vars",
            "CSV/TABLE_subroutine_active_global_vars.csv",
            join=True,
            ref_tables=['subroutines', "user_type_instances", 2, 3],
            selections=['subroutine_id', 'instance_type_id', "", ""],
            conditions=['subroutine_name', 'instance_name'],
            dependency=[0, 1, None, None])

util.insert(cur,
            conn,
            "subroutine_local_arrays",
            "CSV/TABLEs_subroutine_local_array.csv",
            join=True,
            ref_tables=['subroutines', 1,2 ],
            selections=['subroutine_id', '', ""],
            conditions=['subroutine_name', '', ""],
            dependency=[0, None, None])

In [53]:
import re
def TABLE_type_definitions(filename, csv_file='TABLE_type_definitions'):    
    """
    TABLE SETUP
    +--------------+--------------+------+-----+---------+----------------+
    | Field        | Type         | Null | Key | Default | Extra          |
    +--------------+--------------+------+-----+---------+----------------+
    | define_id    | int          | NO   | PRI | NULL    | auto_increment |
    | module_id    | int          | NO   | MUL | 0       |                |
    | user_type_id | int          | NO   | MUL | NULL    |                |
    | member_type  | varchar(100) | NO   |     |         |                |
    | member_name  | varchar(100) | NO   |     | 0       |                |
    | dim          | int          | NO   |     | 0       |                |
    | bounds       | varchar(100) | NO   |     |         |                |
    | active       | varchar(100) | YES  |     | NULL    |                |
    +--------------+--------------+------+-----+---------+----------------+
    """
    cols = ["module_id", "user_type_id", "member_type", "member_name", "dim", "bounds", "active"]
    with open(f"{csv_folder}/{csv_file}.csv", "w") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(cols)
        with open(filename, 'r') as f:
            type = ""
            mod = ""
            for line in f:
                if "Type:" in line:
                    type = line.split()[-1].strip()
                elif "Mod:" in line:
                    mod = line.split()[-1].strip()
                elif "-" in line:
 
                    x = re.findall(r'[^()\s]+|\(.*?\)', line)
                    if len(x) == 2:
                        writer.writerow([mod, type, "Null", "Null", "-1", "Null", "Null"])
                    else:
                        writer.writerow([mod, type, x[1], x[2], x[4], x[3], x[5]])


In [54]:
TABLE_type_definitions("info_files/types_info.txt")


In [84]:
filename = f"{info_files}/types_info.txt"

TABLE_type_definitions("info_files/types_info.txt")

util.insert(cur,
            conn,
            "type_definitions",
            "CSV/TABLE_type_definitions.csv",
            join=True,
            ref_tables=['modules', 'user_types',2, 3, 4, 5, 6 ],
            selections=['module_id', 'user_type_id', "", "", "", "", ""],
            conditions=['module_name', 'user_type_name', "", "", "", "", ""],
            dependency=[0, 1, None, None, None, None, None]
            )